### cis- and trans-QTL mapping with tensorQTL: Modelo Telomeros

Requirements
An environment configured with a GPU and ~50GB of memory.


In [ ]:
import pandas as pd
import torch
import tensorqtl
from tensorqtl import genotypeio, cis, trans
print(f'PyTorch {torch.__version__}')
print(f'Pandas {pd.__version__}')

Three inputs are required for QTL analyses with tensorQTL: genotypes, phenotypes, and covariates.

Phenotypes must be provided in BED format, with a single header line starting with # and the first four columns corresponding to: chr, start, end, phenotype_id, with the remaining columns corresponding to samples (the identifiers must match those in the genotype input). The BED file should specify the center of the cis-window (usually the TSS), with start == end-1.

Covariates can be provided as a tab-delimited text file (covariates x samples) or dataframe (samples x covariates), with row and column headers.

Genotypes must be in PLINK format

In [ ]:
# plink2 \
#     --vcf /TELOMERO/genotipos/Distal.HRC_MAF0.05.PrediXcan_chr1.vcf \
#     --make-bed \
#     --output-chr chrM \
#     --out Distal.HRC_MAF0.05.PrediXcan_chr1

In [ ]:
# Definimos el dataset:
plink_prefix_path = 'GENOTIPO' #se necesitan los archivos bed/bim/fam
expression_bed = 'phenotype_colon_eqtl.bed.gz'
covariates_file = 'archivo_covariables.txt'
prefix = 'eQTLs._samples'

# load phenotypes and covariates
phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expression_bed)
covariates_df = pd.read_csv(covariates_file, sep='\t', index_col=0, low_memory=False).T 

# PLINK reader for genotypes
pr = genotypeio.PlinkReader(plink_prefix_path)
genotype_df = pr.load_genotypes()
variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]

In [ ]:
# map all cis-associations (results for each chromosome are written to file)

# all genes
# cis.map_nominal(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df, prefix)
cis.map_nominal(genotype_df, variant_df, phenotype_df, phenotype_pos_df, prefix, covariates_df=covariates_df)
# genes on chr18
# cis.map_nominal(genotype_df, variant_df,
#                 phenotype_df.loc[phenotype_pos_df['chr']=='chr18'],
#                 phenotype_pos_df.loc[phenotype_pos_df['chr']=='chr18'],
#                 prefix, covariates_df=covariates_df)

In [ ]:
# load results
pairs_df = pd.read_parquet(f'{prefix}.cis_qtl_pairs.chr.parquet')
pairs_df.head()

In [ ]:
# all genes
cis_df = cis.map_cis(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df)

# genes on chr18
# cis_df = cis.map_cis(genotype_df, variant_df, 
#                      phenotype_df.loc[phenotype_pos_df['chr']=='chr18'],
#                      phenotype_pos_df.loc[phenotype_pos_df['chr']=='chr18'],
#                      covariates_df=covariates_df, seed=123456)

In [ ]:
cis_df.head()

In [ ]:
# run mapping
# to limit output size, only associations with p-value <= 1e-5 are returned
trans_df = trans.map_trans(genotype_df, phenotype_df,covariates_df, batch_size=10000,
                           return_sparse=True, pval_threshold=1e-5, maf_threshold=0.05)

In [ ]:
# remove cis-associations
trans_df = trans.filter_cis(trans_df, phenotype_pos_df.T.to_dict(), variant_df, window=5000000)

In [ ]:
trans_df.head(10)

Los archivos que se generan están en formato parquet para pasarlo a un formato de txt:

In [ ]:
import pandas as pd

df = pd.read_parquet('input.parquet')
df.to_csv('out.csv', index=False)             # Exporting to CSV 
df.to_csv('out.txt', index=False, sep=' ')    # Text is just a tad 
df.to_csv('out.txt', index=False, sep='\t')   # Alternatively, you can use tab separators
df.to_json('out.json')                        # Exporting to JSON 

Con los resultados que se obtienen se pasan a MASHR: